# Deep Learning Playground

We are interested in the following features:

- H1: region-averaged time-series
- H2: region-level connectivities (from H1, optional: triu-k1)
- H3: network-averaged time-series (from H1)
- H4: network connectivity (from H3, optional: triu-k1)
- H5: networks connectivity (from H2, optional: triu-k0)

> Note that we are not going to take the upper triangular part of the connectivity matrix and full matrices are used instead.

In [1]:
%reload_ext autoreload
%autoreload 2

import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import RichProgressBar

from src.acnets.deep import ACNetsDataModule, MultiHeadModel

In [2]:
datamodule = ACNetsDataModule(atlas='dosenbach2010', kind='partial correlation', batch_size=8)
datamodule.setup()

n_regions = datamodule.train[0][0].shape[1]
n_networks = datamodule.train[0][2].shape[1]
model = MultiHeadModel(n_regions, n_networks, n_embeddings=8)

trainer = pl.Trainer(accelerator='auto', max_epochs=10000,
                     log_every_n_steps=1, callbacks=[RichProgressBar()])

trainer.fit(model, datamodule)
trainer.test(model, datamodule=datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


┏━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name           ┃ Type               ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ train_accuracy │ MulticlassAccuracy │      0 │
│ 1 │ val_accuracy   │ MulticlassAccuracy │      0 │
│ 2 │ x1_autoencoder │ Seq2SeqAutoEncoder │  117 K │
│ 3 │ x1_head        │ Sequential         │     72 │
│ 4 │ x2_head        │ Sequential         │  204 K │
│ 5 │ x3_autoencoder │ Seq2SeqAutoEncoder │  1.0 K │
│ 6 │ x3_head        │ Sequential         │     72 │
│ 7 │ x4_head        │ Sequential         │    296 │
│ 8 │ x5_head        │ Sequential         │    296 │
│ 9 │ cls_head       │ Sequential         │    218 │
└───┴────────────────┴────────────────────┴────────┘

Trainable params: 323 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 323 K                                                                                                
Total estimated model params size (MB): 1

Output()

/home/morteza/micromamba/envs/acnets/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:54: Detected 
KeyboardInterrupt, attempting graceful shutdown...

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Output()

AssertionError: 

In [ ]:
# Testing Simple Recurrent AutoEncoder

import torch
from torch import nn

n_timesteps = 124
n_networks = 6
batch_size = 8
hidden_size = 6

x = torch.randn(batch_size, n_timesteps, n_networks)

encoder = nn.LSTM(n_networks, hidden_size, batch_first=True)
decoder = nn.LSTM(hidden_size, hidden_size, batch_first=True)

_, (h_enc, c_enc) = encoder(x)
x_dec = torch.randn(batch_size, n_timesteps, hidden_size)
x_recon, _ = decoder(x_dec, (h_enc, c_enc))

x.shape, x_recon.shape

(torch.Size([8, 124, 6]), torch.Size([8, 124, 6]))

In [ ]:
atlases = [
    'dosenbach2010',
    'dosenbach2007',
    'difumo_64_2mm',
    'difumo_128_2mm',
    'difumo_256_2mm',
    'difumo_512_2mm',
    'difumo_1024_2mm',
    'gordon2014_2mm',
    'cort-maxprob-thr25-2mm',
    'seitzman2018',
    'friedman2020',
]

kinds = [
    'correlation',
    'partial correlation',
    'tangent',
    'covariance',
    'precisions'
]